# SLA & Incident Analytics\n
Taruh CSV → run notebook → dapat insight KPI + grafik trend.\n

In [ ]:
import pandas as pd\n
import matplotlib.pyplot as plt\n
REQUIRED = ['incident_id','opened_at','resolved_at','priority','status']\n

In [ ]:
path = 'sample_incidents.csv'  # ganti ke file kamu\n
df = pd.read_csv(path)\n
missing = [c for c in REQUIRED if c not in df.columns]\n
assert not missing, f'Missing required columns: {missing}'\n
df['opened_at'] = pd.to_datetime(df['opened_at'], errors='coerce')\n
df['resolved_at'] = pd.to_datetime(df['resolved_at'], errors='coerce')\n
df['is_resolved'] = df['resolved_at'].notna() & (df['status'].str.lower()=='resolved')\n
df['resolution_minutes'] = (df['resolved_at']-df['opened_at']).dt.total_seconds()/60\n
df.loc[~df['is_resolved'],'resolution_minutes'] = pd.NA\n
df.head()\n

In [ ]:
total=len(df)\n
resolved=int(df['is_resolved'].sum())\n
avg=float(df['resolution_minutes'].dropna().mean()) if resolved else 0\n
kpi={'total_incidents':total,'resolved_incidents':resolved,'avg_resolution_minutes':round(avg,2)}\n
kpi\n

In [ ]:
df['opened_day']=df['opened_at'].dt.date\n
trend=df.groupby('opened_day').size().reset_index(name='incidents')\n
trend\n

In [ ]:
plt.figure()\n
plt.plot(trend['opened_day'].astype(str), trend['incidents'])\n
plt.xticks(rotation=30, ha='right')\n
plt.title('Incident Trend by Day')\n
plt.tight_layout()\n
plt.show()\n

In [ ]:
pd.DataFrame([kpi]).to_csv('kpi_summary.csv', index=False)\n
trend.to_csv('trend_by_day.csv', index=False)\n
print('Exported: kpi_summary.csv, trend_by_day.csv')\n